In [1]:
import boto3
import awswrangler as wr
import pandas as pd
from tqdm import tqdm
accessKeys = pd.read_csv("../../quant-bears_accessKeys.csv")
session = boto3.Session(
	aws_access_key_id=accessKeys.loc[0, "Access key ID"],
	aws_secret_access_key=accessKeys.loc[0, "Secret access key"]
)

s3_collection_path = "s3://quant-bears-data-collection/raw-data/"
s3_price_collection_path = "s3://quant-bears-data-collection/raw-resolved-price/"

In [3]:
today = "2023-10-18"
previousDay = "2023-10-11"

prev_df = wr.s3.read_parquet(s3_collection_path + f"seekingAlpha.seekingAlphaBulkMetrics/{previousDay}.parquet", boto3_session=session)

In [4]:
prev_df.head()

,ticker,altman_z_score,analysts_down,analysts_down_percent,analysts_down_percent_avg_5y,analysts_up,analysts_up_percent,analysts_up_percent_avg_5y,assets_turnover,assets_turnover_avg_5y,...,total_debt,total_revenue,working_cap_change,working_cap_change_avg_5y,yld_on_cost_1y,yld_on_cost_1y_avg_5y,yld_on_cost_3y,yld_on_cost_3y_avg_5y,yld_on_cost_5y,yld_on_cost_5y_avg_5y
0,A,6.153144,15.0,100.000000,13.478062,0.0,0.000000,86.521940,0.661089,0.602098,...,2.967000e+09,6.994000e+09,26.9574,-0.412132,0.714569,0.799212,0.850983,1.197979,1.349933,1.474432
1,AA,1.401235,7.0,100.000000,60.595860,0.0,0.000000,39.404140,0.728491,0.757523,...,1.834000e+09,1.086800e+10,-21.0507,19.992653,1.029866,NaN,3.194888,NaN,1.109262,NaN
2,AAL,0.862385,13.0,76.470590,54.442814,4.0,23.529411,45.557186,0.782322,0.598929,...,4.258300e+10,5.289400e+10,43.6042,-158.888410,NaN,NaN,NaN,NaN,NaN,NaN
3,AAP,2.266178,16.0,100.000000,39.881680,0.0,0.000000,60.118320,0.917976,0.928705,...,4.596040e+09,1.121875e+10,54.5253,-10.168253,2.857143,1.237327,3.058203,1.596553,2.911074,1.837003
4,AAPL,7.639575,8.0,21.621622,24.091553,29.0,78.378380,75.908450,1.143769,0.932244,...,1.092800e+11,3.839330e+11,-86.8950,-59.327420,0.669420,1.778697,0.803625,2.862522,1.737844,3.806725


In [6]:
prev_df = prev_df.set_index("ticker")

In [7]:
resolved_price = wr.s3.read_parquet(s3_price_collection_path + f"{today}.parquet", boto3_session=session)

In [9]:
resolved_price = resolved_price.set_index("ticker")

In [14]:
prev_df["primary_price"].corr(resolved_price["primary_price"])

0.9999999667709722

In [15]:
prev_df.loc["AAPL"]["primary_price"]

178.39

In [16]:
price_change = (resolved_price["primary_price"] - prev_df["primary_price"]) / prev_df["primary_price"]

In [37]:
resolved_price["primary_price"]

ticker
AEO      18.20
AAPL    177.15
AEE      78.40
AEP      75.67
AES      13.75
         ...  
WRBY     14.80
VTYX     17.49
VSTS     15.89
ZWS      27.04
ZUO       8.25
Name: primary_price, Length: 1599, dtype: float64

In [38]:
prev_df["primary_price"]

ticker
A       113.29
AA       28.20
AAL      12.26
AAP      52.09
AAPL    178.39
         ...  
ZS      171.88
ZTO      24.63
ZTS     175.63
ZUO       7.98
ZWS      26.81
Name: primary_price, Length: 1602, dtype: float64

In [21]:
(1. + price_change["AAPL"]) * 1_000_000

993048.9377207244

In [28]:
resolved_price.index.shape

(1599,)

In [29]:
prev_df.shape

(1602, 264)

In [32]:
import pandas_market_calendars as mcal
import datetime

def getPreviousMarketOpenDays(date: str, n: int):
	# Create a calendar
	nyse = mcal.get_calendar('NYSE')
	day = datetime.datetime.strptime(date, "%Y-%m-%d")
	schedule = nyse.schedule(
		start_date = (day - datetime.timedelta(days = 20 + n*2)).strftime("%Y-%m-%d"),
		end_date=day.strftime("%Y-%m-%d"))
	
	return schedule.index.strftime("%Y-%m-%d").tolist()[-n:]

In [35]:
resolved_price

,primary_price
ticker,
AEO,18.20
AAPL,177.15
AEE,78.40
AEP,75.67
AES,13.75
...,...
WRBY,14.80
VTYX,17.49
VSTS,15.89


In [36]:
prev_df["primary_price"].isna().sum()

1